# Final Project Notebook

The report for this final project can be found at this [link](https://cybertraining-dsc.github.io/report/fa20-523-301/project/project/).

## Part 1 Importing the functions

This file requires that we import Numpy, Matplotlib, Pylab, Keras, and Pandas

In [1]:
import numpy as np
from numpy import mean
from numpy import std

import matplotlib.pyplot as plt
import pylab
import os, sys
import pandas as pd
import io
import requests
import warnings
import sklearn
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from datetime import datetime


from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from matplotlib import pyplot

from sklearn.datasets import make_regression

! pip install cloudmesh-common -U

from cloudmesh.common.Benchmark import Benchmark

! pip install utils
! pip install lightgbm
import utils
import lightgbm

from google.colab import drive
drive.mount('/content/drive')

     |████████████████████████████████| 81kB 2.1MB/s 
     |████████████████████████████████| 133kB 4.2MB/s 
  Created wheel for python-hostlist: filename=python_hostlist-1.21-cp36-none-any.whl size=38932 sha256=dd7c91f30fae25799bed32b58117c29cfaa64c0fbef09660eb1935ff41dd6f73
  Stored in directory: /root/.cache/pip/wheels/0b/5b/55/ddcf52288f0b10f4564ca1b2531594ff7ccc65f487ba8dc437
Successfully built python-hostlist
Mounted at /content/drive


## Conditioned Code. Only Use this for work to save on processing.

Moved all of the files to the following [Drive Location](https://drive.google.com/drive/folders/1flPbO7Q0ID70Or_OZ4MdW6CbYDE32GeT). 

In [2]:
########################################################################
# Load In Conditioned Datasets - Upload to the Colab files on the left #
########################################################################

# df_Injury_End
df_Injury_End = pd.read_csv("/content/drive/My Drive/Colab Notebooks/df_Injury_End.csv")  

#df_Injury_End = pd.read_csv("/content/drive/Shared with me/Colab Notebooks/df_Injury_End.csv")  
# df_Injury_length
df_Injury_length = pd.read_csv('/content/drive/My Drive/Colab Notebooks/df_Injury_length.csv') 
# df_Injury_Start
df_Injury_Start = pd.read_csv('/content/drive/My Drive/Colab Notebooks/df_Injury_Start.csv') 

ds_NBA_Injuries = [df_Injury_Start, df_Injury_End]
#nba games dataset
df_Games_games = pd.read_csv('/content/drive/My Drive/Colab Notebooks/games.csv')
df_Games_gamesDetails = pd.read_csv('/content/drive/My Drive/Colab Notebooks/games_details.csv')
df_Games_players = pd.read_csv('/content/drive/My Drive/Colab Notebooks/players.csv')
df_Games_ranking = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ranking.csv')
df_Games_teams = pd.read_csv('/content/drive/My Drive/Colab Notebooks/teams.csv')
ds_NBA_Games = [df_Games_games, df_Games_gamesDetails, df_Games_players, df_Games_ranking, df_Games_teams]

In [3]:
df_Injury_End.head()

,Unnamed: 0,Date,Team,Acquired,Relinquished,Notes,PLAYER_ID,TEAM_ID,NICKNAME,inj_MIN,inj_FGA,inj_FG_PCT,inj_FG3A,inj_FG3_PCT,inj_FTA,inj_FT_PCT,inj_REB,inj_AST,inj_STL,inj_BLK,inj_TO,inj_PF,inj_PTS,inj_PLUS_MINUS,p5_MIN,p5_FGA,p5_FG_PCT,p5_FG3A,p5_FG3_PCT,p5_FTA,p5_FT_PCT,p5_REB,p5_AST,p5_STL,p5_BLK,p5_TO,p5_PF,p5_PTS,p5_PLUS_MINUS
0,0,2010-10-27,Heat,Jerry Stackhouse,NaN,activated from IL,711.0,1610612748,Heat,5.0,7.0,0.714,2.0,0.5,2.0,1.000,2.0,6.0,1.0,0.0,0.0,2.0,13.0,NaN,29.590000,12.6,0.3816,2.8,0.4666,6.4,0.6654,3.6,4.6,1.2,0.0,2.8,2.0,15.8,6.75
1,1,2010-10-29,Heat,Jamaal Magloire,NaN,activated from IL,2048.0,1610612748,Heat,5.0,5.0,0.400,0.0,0.0,6.0,0.667,7.0,1.0,1.0,0.0,3.0,3.0,8.0,NaN,30.606667,8.6,0.4712,0.0,0.0000,2.8,0.3666,6.4,0.6,0.8,0.8,2.4,3.4,10.6,-5.60
2,2,2010-10-29,Heat,Mario Chalmers,NaN,returned to lineup,201596.0,1610612748,Heat,24.0,3.0,0.333,2.0,0.5,2.0,0.500,3.0,3.0,2.0,0.0,4.0,5.0,4.0,NaN,22.000000,5.8,0.4332,3.0,0.2166,1.8,0.3334,3.2,3.4,1.0,0.0,1.8,3.8,7.2,NaN
3,3,2010-11-02,Heat,Juwan Howard,NaN,activated from IL,436.0,1610612748,Heat,5.0,2.0,0.500,0.0,0.0,4.0,1.000,3.0,0.0,1.0,0.0,3.0,3.0,6.0,NaN,30.100000,12.6,0.3678,0.2,0.0000,4.8,0.7666,8.6,1.4,0.8,0.2,2.4,3.2,12.4,-2.75
4,4,2010-11-06,Heat,Jamaal Magloire,NaN,activated from IL,2048.0,1610612748,Heat,5.0,5.0,0.400,0.0,0.0,6.0,0.667,7.0,1.0,1.0,0.0,3.0,3.0,8.0,NaN,30.606667,8.6,0.4712,0.0,0.0000,2.8,0.3666,6.4,0.6,0.8,0.8,2.4,3.4,10.6,-5.60


In [4]:
df_Injury_length.head()

,Unnamed: 0,TEAM_ID,Team,PLAYER_ID,Player,DateInjured,InjuryNotes,DateRecovered,RecoverNotes
0,0,1610612741,Bulls,2430.0,Carlos Boozer,2010-10-03,fractured bone in right pinky finger (out inde...,2010-12-01,activated from IL
1,1,1610612741,Bulls,2430.0,Carlos Boozer,2010-10-27,placed on IL with fractured bone in right pink...,2010-12-01,activated from IL
2,2,1610612741,Bulls,201959.0,Taj Gibson,2010-11-24,right ankle/foot injury (DNP),2010-12-22,activated from IL
3,3,1610612741,Bulls,201565.0,Derrick Rose,2010-11-26,placed on IL with sore neck,2010-11-27,activated from IL
4,4,1610612741,Bulls,101249.0,John Lucas III,2010-11-27,placed on IL,2010-12-21,activated from IL


In [5]:
df_Injury_Start.head()

,Unnamed: 0,Date,Team,Acquired,Relinquished,Notes,PLAYER_ID,TEAM_ID,NICKNAME,inj_MIN,inj_FGA,inj_FG_PCT,inj_FG3A,inj_FG3_PCT,inj_FTA,inj_FT_PCT,inj_REB,inj_AST,inj_STL,inj_BLK,inj_TO,inj_PF,inj_PTS,inj_PLUS_MINUS,p5_MIN,p5_FGA,p5_FG_PCT,p5_FG3A,p5_FG3_PCT,p5_FTA,p5_FT_PCT,p5_REB,p5_AST,p5_STL,p5_BLK,p5_TO,p5_PF,p5_PTS,p5_PLUS_MINUS
0,0,2010-10-03,Bulls,NaN,Carlos Boozer,fractured bone in right pinky finger (out inde...,2430.0,1610612741,Bulls,36.766667,11.0,0.364,0.0,0.0,2.0,1.0,14.0,0.0,0.0,0.0,4.0,6.0,10.0,-14.0,40.936667,17.6,0.5420,0.0,0.0000,2.0,0.4334,14.8,3.4,0.4,0.4,2.6,3.6,19.8,-5.0
1,1,2010-10-27,Bulls,NaN,Carlos Boozer,placed on IL with fractured bone in right pink...,2430.0,1610612741,Bulls,36.766667,11.0,0.364,0.0,0.0,2.0,1.0,14.0,0.0,0.0,0.0,4.0,6.0,10.0,-14.0,40.936667,17.6,0.5420,0.0,0.0000,2.0,0.4334,14.8,3.4,0.4,0.4,2.6,3.6,19.8,-5.0
2,2,2010-11-24,Bulls,NaN,Taj Gibson,right ankle/foot injury (DNP),201959.0,1610612741,Bulls,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.876667,9.6,0.3554,0.4,0.2000,1.6,0.4000,9.6,1.4,1.0,2.0,1.4,3.0,8.8,-4.0
3,3,2010-11-26,Bulls,NaN,Derrick Rose,placed on IL with sore neck,201565.0,1610612741,Bulls,50.316667,33.0,0.424,5.0,0.2,6.0,1.0,12.0,7.0,0.0,1.0,4.0,6.0,35.0,7.0,38.500000,21.8,0.5034,4.2,0.4566,6.4,0.7446,3.8,6.6,1.4,0.4,3.4,1.6,28.4,1.2
4,4,2010-11-27,Bulls,NaN,John Lucas III,placed on IL,101249.0,1610612741,Bulls,4.416667,2.0,0.500,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,6.400000,3.2,0.2734,1.6,0.2000,0.0,0.0000,0.4,0.8,0.0,0.0,0.2,0.6,2.2,NaN


In [ ]:
# ########################################################################
# # Load In Conditioned Datasets - Upload to the Colab files on the left #
# ########################################################################

# # df_Injury_End
# df_Injury_End = pd.read_csv("df_Injury_End.csv")  
# # df_Injury_length
# df_Injury_length = pd.read_csv('df_Injury_length.csv') 
# # df_Injury_Start
# df_Injury_Start = pd.read_csv('df_Injury_Start.csv') 

# ds_NBA_Injuries = [df_Injury_Start, df_Injury_End]
# #nba games dataset
# df_Games_games = pd.read_csv('games.csv')
# df_Games_gamesDetails = pd.read_csv('games_details.csv')
# df_Games_players = pd.read_csv('players.csv')
# df_Games_ranking = pd.read_csv('ranking.csv')
# df_Games_teams = pd.read_csv('teams.csv')
# ds_NBA_Games = [df_Games_games, df_Games_gamesDetails, df_Games_players, df_Games_ranking, df_Games_teams]

## The following is what to use if the data hasn't been conditioned.

Now that the funtions have been imported the team can focus on the download coding. The following cells will set up an install for Kaggle files and prompt for an upload of the kaggle.json file for credentials. 

The mkdir function creates a directory for the Kaggle data. This cell will allow the team to verify that the kaggle.json file appropriately uploaded to the directory.

In [ ]:
##import the kaggle.json from local to drive
!pip install -q kaggle
from google.colab import files
##when it asks you to choose a file select the kaggle.json located within the 'project' folder from the github repo
files.upload()

In [ ]:
##make a kaggle and a data folder
!mkdir ~/.kaggle
!mkdir data
##copy the kaggle.json to the .kaggle folder then grant permissions
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
#test to see if kaggle is working, should print list of datasets
!kaggle datasets list

Now, the team must download all of the datasets for the class. The three datasets are focused on the NBA. 

The first dataset is for injuries. Each injury will be used to set up players, timeframes, and severity of injuries. 

The other two datasets are for the player performance. By cross referencing this data to the previous list, the team will be able to see which players are limited from the injury and how performance is hampered by time in rehab.

In [ ]:
##downloading all the datasets
!kaggle datasets download -d ghopkins/nba-injuries-2010-2018
!kaggle datasets download -d nathanlauga/nba-games
!kaggle datasets download -d pablote/nba-enhanced-stats
##unzipping to the data folder
!unzip nba-injuries-2010-2018.zip -d data
!unzip nba-games.zip -d data
!unzip nba-enhanced-stats.zip -d data

In [ ]:
########################################################################
# Load In Conditioned Datasets - Upload to the Colab files on the left #
########################################################################

# df_Injury_End
df_Injury_End = pd.read_csv("df_Injury_End.csv")  
# df_Injury_length
df_Injury_length = pd.read_csv('df_Injury_length.csv') 
# df_Injury_Start
df_Injury_Start = pd.read_csv('df_Injury_Start.csv') 

ds_NBA_Injuries = [df_Injury_Start, df_Injury_End]
#nba games dataset
df_Games_games = pd.read_csv('games.csv')
df_Games_gamesDetails = pd.read_csv('games_details.csv')
df_Games_players = pd.read_csv('players.csv')
df_Games_ranking = pd.read_csv('ranking.csv')
df_Games_teams = pd.read_csv('teams.csv')
ds_NBA_Games = [df_Games_games, df_Games_gamesDetails, df_Games_players, df_Games_ranking, df_Games_teams]

#ADD Images fo data engineering and exploratory analysis


## We are now ready to structure our dataframes

The team must now use these downloads to create dataframes. Pandas dataframes will be easier to manage the data. The team will be able to use Pandas to process the data and allow the team to make correlations for feature engineering to create the models.

# Part 2 Building the Models

A link for Keras for us to use can be found [here](https://keras.io/guides/sequential_model/). We are first going to set up our Benchmark Test to be used when we are Benchmarking our model.

In [6]:
def b():
  Benchmark.Start()
  print ("b")
  import time
  time.sleep(3)
  Benchmark.Stop()

def c():
  Benchmark.Start()
  print ("c")
  import time
  time.sleep(1)
  Benchmark.Stop()

In [7]:
 b()
 c()

 Benchmark.print()

b
c

+---------------------+------------------------------------------------------------------+
| Attribute           | Value                                                            |
|---------------------+------------------------------------------------------------------|
| BUG_REPORT_URL      | "https://bugs.launchpad.net/ubuntu/"                             |
| DISTRIB_CODENAME    | bionic                                                           |
| DISTRIB_DESCRIPTION | "Ubuntu 18.04.5 LTS"                                             |
| DISTRIB_ID          | Ubuntu                                                           |
| DISTRIB_RELEASE     | 18.04                                                            |
| HOME_URL            | "https://www.ubuntu.com/"                                        |
| ID                  | ubuntu                                                           |
| ID_LIKE             | debian                                                       

Now that we know which GPU we are using, we can get into the actual work. The following is building our Keras model.

In [8]:
np.random.seed(23)
warnings.filterwarnings("ignore")

In [9]:
df_Injury_End_fillna = df_Injury_End.copy()

In [10]:
cols = ['Date', 'Team', 'Acquired', 'Relinquished', 'Notes', 'TEAM_ID', 'NICKNAME']
df_Injury_End_fillna = df_Injury_End_fillna.drop(cols, 1)

In [12]:
############## Need to work here more. The Fill NA command isn't taking out NaNs ############

df_Injury_End_fillna = df_Injury_End_fillna.fillna(df_Injury_End_fillna.mean())

In [13]:
df_Injury_End_fillna.head()

,Unnamed: 0,PLAYER_ID,inj_MIN,inj_FGA,inj_FG_PCT,inj_FG3A,inj_FG3_PCT,inj_FTA,inj_FT_PCT,inj_REB,inj_AST,inj_STL,inj_BLK,inj_TO,inj_PF,inj_PTS,inj_PLUS_MINUS,p5_MIN,p5_FGA,p5_FG_PCT,p5_FG3A,p5_FG3_PCT,p5_FTA,p5_FT_PCT,p5_REB,p5_AST,p5_STL,p5_BLK,p5_TO,p5_PF,p5_PTS,p5_PLUS_MINUS
0,0,711.0,5.0,7.0,0.714,2.0,0.5,2.0,1.000,2.0,6.0,1.0,0.0,0.0,2.0,13.0,-1.3182,29.590000,12.6,0.3816,2.8,0.4666,6.4,0.6654,3.6,4.6,1.2,0.0,2.8,2.0,15.8,6.750000
1,1,2048.0,5.0,5.0,0.400,0.0,0.0,6.0,0.667,7.0,1.0,1.0,0.0,3.0,3.0,8.0,-1.3182,30.606667,8.6,0.4712,0.0,0.0000,2.8,0.3666,6.4,0.6,0.8,0.8,2.4,3.4,10.6,-5.600000
2,2,201596.0,24.0,3.0,0.333,2.0,0.5,2.0,0.500,3.0,3.0,2.0,0.0,4.0,5.0,4.0,-1.3182,22.000000,5.8,0.4332,3.0,0.2166,1.8,0.3334,3.2,3.4,1.0,0.0,1.8,3.8,7.2,-0.825948
3,3,436.0,5.0,2.0,0.500,0.0,0.0,4.0,1.000,3.0,0.0,1.0,0.0,3.0,3.0,6.0,-1.3182,30.100000,12.6,0.3678,0.2,0.0000,4.8,0.7666,8.6,1.4,0.8,0.2,2.4,3.2,12.4,-2.750000
4,4,2048.0,5.0,5.0,0.400,0.0,0.0,6.0,0.667,7.0,1.0,1.0,0.0,3.0,3.0,8.0,-1.3182,30.606667,8.6,0.4712,0.0,0.0000,2.8,0.3666,6.4,0.6,0.8,0.8,2.4,3.4,10.6,-5.600000


In [14]:
df_baseline = df_Injury_End_fillna.copy()
sort_by = 'PLAYER_ID'

#df_baseline.sort_values(by=['Date','Name']).reset_index(drop=True, inplace=True)
df_baseline.sort_values(by=[sort_by]).reset_index(drop=True, inplace=True)
# df_baseline['FPTS_pred'] = utils.calculate_FPTS(df_baseline)

# # Season average
# print(' MAE | ', utils.calculate_MAE(df_baseline['FPTS_pred'], df_baseline['FPTS']))
# print('RMSE | ', utils.calculate_RMSE(df_baseline['FPTS_pred'], df_baseline['FPTS']))

In [15]:
df_baseline.head()

,Unnamed: 0,PLAYER_ID,inj_MIN,inj_FGA,inj_FG_PCT,inj_FG3A,inj_FG3_PCT,inj_FTA,inj_FT_PCT,inj_REB,inj_AST,inj_STL,inj_BLK,inj_TO,inj_PF,inj_PTS,inj_PLUS_MINUS,p5_MIN,p5_FGA,p5_FG_PCT,p5_FG3A,p5_FG3_PCT,p5_FTA,p5_FT_PCT,p5_REB,p5_AST,p5_STL,p5_BLK,p5_TO,p5_PF,p5_PTS,p5_PLUS_MINUS
0,0,711.0,5.0,7.0,0.714,2.0,0.5,2.0,1.000,2.0,6.0,1.0,0.0,0.0,2.0,13.0,-1.3182,29.590000,12.6,0.3816,2.8,0.4666,6.4,0.6654,3.6,4.6,1.2,0.0,2.8,2.0,15.8,6.750000
1,1,2048.0,5.0,5.0,0.400,0.0,0.0,6.0,0.667,7.0,1.0,1.0,0.0,3.0,3.0,8.0,-1.3182,30.606667,8.6,0.4712,0.0,0.0000,2.8,0.3666,6.4,0.6,0.8,0.8,2.4,3.4,10.6,-5.600000
2,2,201596.0,24.0,3.0,0.333,2.0,0.5,2.0,0.500,3.0,3.0,2.0,0.0,4.0,5.0,4.0,-1.3182,22.000000,5.8,0.4332,3.0,0.2166,1.8,0.3334,3.2,3.4,1.0,0.0,1.8,3.8,7.2,-0.825948
3,3,436.0,5.0,2.0,0.500,0.0,0.0,4.0,1.000,3.0,0.0,1.0,0.0,3.0,3.0,6.0,-1.3182,30.100000,12.6,0.3678,0.2,0.0000,4.8,0.7666,8.6,1.4,0.8,0.2,2.4,3.2,12.4,-2.750000
4,4,2048.0,5.0,5.0,0.400,0.0,0.0,6.0,0.667,7.0,1.0,1.0,0.0,3.0,3.0,8.0,-1.3182,30.606667,8.6,0.4712,0.0,0.0000,2.8,0.3666,6.4,0.6,0.8,0.8,2.4,3.4,10.6,-5.600000


In [16]:
#df_baseline.sort_values(by=['Date','Name']).reset_index(drop=True, inplace=True)
df_baseline.sort_values(by=[sort_by]).reset_index(drop=True, inplace=True)


# df_baseline['	PLAYER_ID'] = utils.calculate_FPTS(df_baseline)

# # Season average
# print(' MAE | ', utils.calculate_MAE(df_baseline['FPTS_pred'], df_baseline['FPTS']))
# print('RMSE | ', utils.calculate_RMSE(df_baseline['FPTS_pred'], df_baseline['FPTS']))

# Part No X. Building the Model

The team is now moving on to building the model for the baseline. Linear Regression can be used to model the values. Additionally, a Random Forest modeling function was used to verify model performance. 

The pipeline will be built off of using df_baseline.p5_PLUS_MINUS. To change the data going into the model, the team had to modify the dataframe input to get to the results. Comments were put around to make it easy to find the code in the Notebook.

In [17]:
#df_baseline = df_baseline
# basic =  ['PTS','3P','AST','TRB','STL','BLK','TOV', 'DD', 'TD']

###################################
##                               ##
##     INSERT CODE               ##
##                               ##
##     Change DF before here     ##
##                               ##
##                               ##
##                               ##
###################################

X = df_baseline

In [18]:
X.head()

,Unnamed: 0,PLAYER_ID,inj_MIN,inj_FGA,inj_FG_PCT,inj_FG3A,inj_FG3_PCT,inj_FTA,inj_FT_PCT,inj_REB,inj_AST,inj_STL,inj_BLK,inj_TO,inj_PF,inj_PTS,inj_PLUS_MINUS,p5_MIN,p5_FGA,p5_FG_PCT,p5_FG3A,p5_FG3_PCT,p5_FTA,p5_FT_PCT,p5_REB,p5_AST,p5_STL,p5_BLK,p5_TO,p5_PF,p5_PTS,p5_PLUS_MINUS
0,0,711.0,5.0,7.0,0.714,2.0,0.5,2.0,1.000,2.0,6.0,1.0,0.0,0.0,2.0,13.0,-1.3182,29.590000,12.6,0.3816,2.8,0.4666,6.4,0.6654,3.6,4.6,1.2,0.0,2.8,2.0,15.8,6.750000
1,1,2048.0,5.0,5.0,0.400,0.0,0.0,6.0,0.667,7.0,1.0,1.0,0.0,3.0,3.0,8.0,-1.3182,30.606667,8.6,0.4712,0.0,0.0000,2.8,0.3666,6.4,0.6,0.8,0.8,2.4,3.4,10.6,-5.600000
2,2,201596.0,24.0,3.0,0.333,2.0,0.5,2.0,0.500,3.0,3.0,2.0,0.0,4.0,5.0,4.0,-1.3182,22.000000,5.8,0.4332,3.0,0.2166,1.8,0.3334,3.2,3.4,1.0,0.0,1.8,3.8,7.2,-0.825948
3,3,436.0,5.0,2.0,0.500,0.0,0.0,4.0,1.000,3.0,0.0,1.0,0.0,3.0,3.0,6.0,-1.3182,30.100000,12.6,0.3678,0.2,0.0000,4.8,0.7666,8.6,1.4,0.8,0.2,2.4,3.2,12.4,-2.750000
4,4,2048.0,5.0,5.0,0.400,0.0,0.0,6.0,0.667,7.0,1.0,1.0,0.0,3.0,3.0,8.0,-1.3182,30.606667,8.6,0.4712,0.0,0.0000,2.8,0.3666,6.4,0.6,0.8,0.8,2.4,3.4,10.6,-5.600000


In [19]:
X = X.values.reshape(-1, 1)
# X = X.reshape(-1, 1)


# X = df_baseline.loc[:, basic]
X = MinMaxScaler().fit_transform(X)
print(X.shape)
# Y = df_baseline['FPTS'].values.reshape(-1,1).flatten()
Y = df_baseline.values.reshape(-1,1).flatten() # Y is 38 times larger. Not sure what I did here. 
Y = Y.reshape(-1, 1) 

size_x = X.shape[0]
size_y = Y.shape[0]
size_y = int(size_y/size_x)
print(size_y)

Y = Y.reshape((size_x, size_y)) # Y is 29 times larger. Not sure what I did here. 



print(X.shape)
print(Y.shape)


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=101)


lin_reg = LinearRegression()
# rf=RandomForestClassifier(max_depth=8,n_estimators=5)

# scores = cross_validate(lasso, X, Y, cv=3, scoring=('r2', 'neg_mean_squared_error'), return_train_score=True)


reg_cv_score=cross_val_score(estimator=lin_reg,X=X_train,y=Y_train,cv=5)
print(reg_cv_score) # Prints a bunch of NaNs currently.

# errors = utils.cross_val(reg, X, y, n_folds=5, verbose=0)
# utils.summarize_errors(errors)

(295040, 1)
1
(295040, 1)
(295040, 1)
[1. 1. 1. 1. 1.]


The model has been built and trained on 2/3 of the data with a test on 1/3 of the data. 

In [20]:
# When the dataframes are combined, use this code to select features.

# features = ['SG', 'F', 'C', 'PTS', '3P', 'AST', 'TRB', 'STL', 'BLK', 'TOV', 'DD', 'TD', 'MP', 'FT',
#             'FTA', 'FGA', '3PA', 'DRB', 'ORB', 'USG_perc', 'DRtg', 'ORtg', 'AST_perc', 'DRB_perc',
#             'ORB_perc', 'BLK_perc', 'TOV_perc', 'STL_perc', 'eFG_perc', 'FG_perc', '3P_perc', 'FT_perc']


features = ['inj_FGA',	'inj_FG_PCT',	'inj_FG3A',	'inj_FG3_PCT',	'inj_FTA',	'inj_FT_PCT',	'inj_REB',	'inj_AST',	'inj_STL',	
            'inj_BLK',	'inj_TO',	'inj_PF', 'inj_PTS',	'inj_PLUS_MINUS',	'p5_MIN',	
            'p5_FGA',	'p5_FG_PCT', 	'p5_FG3A', 	'p5_FG3_PCT',	'p5_FTA',	'p5_FT_PCT',	'p5_REB',
            'p5_AST',	'p5_STL',	'p5_BLK',	'p5_TO',	'p5_PF',	'p5_PTS',	'p5_PLUS_MINUS']

In [24]:
# _all = ['Salary', 'Rest', 'Rota_All', 'Rota_Pos', 'Home', 'SG', 'F', 'C', 'Value', 'FPTS_std',
#         'PTS', '3P', 'AST', 'TRB', 'STL', 'BLK', 'TOV', 'DD', 'TD', 'MP', 'FT', 'FTA', 'FGA', '3PA', 'DRB',
#         'ORB', 'USG_perc', 'DRtg', 'ORtg', 'AST_perc', 'DRB_perc', 'ORB_perc', 'BLK_perc', 'TOV_perc', 
#         'STL_perc', 'eFG_perc', 'FG_perc', '3P_perc', 'FT_perc']
_all  = ['inj_FGA',	'inj_FG_PCT',	'inj_FG3A',	'inj_FG3_PCT',	'inj_FTA',	'inj_FT_PCT',	'inj_REB',	'inj_AST',	'inj_STL',	
            'inj_BLK',	'inj_TO',	'inj_PF', 'inj_PTS',	'inj_PLUS_MINUS',	'p5_MIN',	
            'p5_FGA',	'p5_FG_PCT', 	'p5_FG3A', 	'p5_FG3_PCT',	'p5_FTA',	'p5_FT_PCT',	'p5_REB',
            'p5_AST',	'p5_STL',	'p5_BLK',	'p5_TO',	'p5_PF',	'p5_PTS',	'p5_PLUS_MINUS']

In [25]:
df_Injury_End_fillna.head()

,Unnamed: 0,PLAYER_ID,inj_MIN,inj_FGA,inj_FG_PCT,inj_FG3A,inj_FG3_PCT,inj_FTA,inj_FT_PCT,inj_REB,inj_AST,inj_STL,inj_BLK,inj_TO,inj_PF,inj_PTS,inj_PLUS_MINUS,p5_MIN,p5_FGA,p5_FG_PCT,p5_FG3A,p5_FG3_PCT,p5_FTA,p5_FT_PCT,p5_REB,p5_AST,p5_STL,p5_BLK,p5_TO,p5_PF,p5_PTS,p5_PLUS_MINUS
0,0,711.0,5.0,7.0,0.714,2.0,0.5,2.0,1.000,2.0,6.0,1.0,0.0,0.0,2.0,13.0,-1.3182,29.590000,12.6,0.3816,2.8,0.4666,6.4,0.6654,3.6,4.6,1.2,0.0,2.8,2.0,15.8,6.750000
1,1,2048.0,5.0,5.0,0.400,0.0,0.0,6.0,0.667,7.0,1.0,1.0,0.0,3.0,3.0,8.0,-1.3182,30.606667,8.6,0.4712,0.0,0.0000,2.8,0.3666,6.4,0.6,0.8,0.8,2.4,3.4,10.6,-5.600000
2,2,201596.0,24.0,3.0,0.333,2.0,0.5,2.0,0.500,3.0,3.0,2.0,0.0,4.0,5.0,4.0,-1.3182,22.000000,5.8,0.4332,3.0,0.2166,1.8,0.3334,3.2,3.4,1.0,0.0,1.8,3.8,7.2,-0.825948
3,3,436.0,5.0,2.0,0.500,0.0,0.0,4.0,1.000,3.0,0.0,1.0,0.0,3.0,3.0,6.0,-1.3182,30.100000,12.6,0.3678,0.2,0.0000,4.8,0.7666,8.6,1.4,0.8,0.2,2.4,3.2,12.4,-2.750000
4,4,2048.0,5.0,5.0,0.400,0.0,0.0,6.0,0.667,7.0,1.0,1.0,0.0,3.0,3.0,8.0,-1.3182,30.606667,8.6,0.4712,0.0,0.0000,2.8,0.3666,6.4,0.6,0.8,0.8,2.4,3.4,10.6,-5.600000


In [26]:
df_baseline = df_Injury_End_fillna.copy()
sort_by = 'PLAYER_ID'

In [27]:
### Keeps Passing error of ValueError: Input contains NaN, infinity or a value too large for dtype('float32').
### This should work when we start passing numbers. It will help pick best features.

def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)




# df_baseline = clean_dataset(df_baseline)
# #df_baseline.sort_values(by=['Date','Name']).reset_index(drop=True, inplace=True)
# df_baseline.sort_values(by=[sort_by]).reset_index(drop=True, inplace=True)

# print(df_baseline.head)

X = df_baseline
X = X.values.reshape(-1, 1)
# X was called above

# X = MinMaxScaler().fit_transform(X)
# y = df_features['FPTS'].values.reshape(-1,1).flatten()
print(X)

Y = df_baseline.values.reshape(-1,1).flatten() # Y is 38 times larger. Not sure what I did here. 
Y = Y.reshape(-1, 1) 

print(X.shape)
print(Y.shape)

size_x = X.shape[0]
size_y = Y.shape[0]
size_y = int(size_y/size_x)
print(size_y)

Y = Y.reshape((size_x, size_y))
# Y = Y.reshape((16894, 38)) # Y is 38 times larger. Not sure what I did here. 

# Takes 2 minutes
# clf.set_params(n_estimators=2000)
# clf.fit(X, y, sample_weight=train_weight)

model = GradientBoostingRegressor()
model.fit(X, Y)

top_features = pd.Series(model.feature_importances_, index = _all).sort_values()
top_features.plot(kind = "barh", figsize=(15,10) ,title='Top Features')
plt.show()

[[  0.  ]
 [711.  ]
 [  5.  ]
 ...
 [  2.5 ]
 [ 24.75]
 [  3.5 ]]
(295040, 1)
(295040, 1)
1


ValueError: ignored

In [29]:

# lightgbm for regression

X = df_baseline
# X_scrub = df_baseline

# mean_X = np.mean(X_scrub)
# for i in  range(len(X_scrub)):
#   # print(mean_X)
#   X[i] = X_scrub[i]

# X = X.values.reshape(-1, 1)
X = X.values.reshape(-1, 1)

Y = df_baseline.values.reshape(-1,1).flatten() # Y is 38 times larger. Not sure what I did here. 
Y = Y.reshape(-1, 1) 

size_x = X.shape[0]
size_y = Y.shape[0]
size_y = int(size_y/size_x)
print(size_y)
Y = Y.reshape((size_x, size_y))

print("X Shape is ",X.shape)
print("Y Shape is ",Y.shape)

# Neat code from other write-up. This creates a dataset to define dataset
# X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, random_state=1)

# evaluate the model
model = LGBMRegressor()
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

### Searched Online. I need to use the TfidfVectorizer
# from sklearn.feature_extraction.text import TfidfVectorizer
# tfidf = TfidfVectorizer(binary=True)
# Y = tfidf.fit_transform(Y)

print("X Shape is ",X.shape)
print("Y Shape is ",Y.shape)

# print("Y Shape is ",Y.shape)

# n_scores = cross_val_score(model, X, Y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
# print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

# fit the model on the whole dataset
# model = LGBMRegressor()
model.fit(X, Y)

# make a single prediction. Code example is below:
# row = [[2.02220122, 0.31563495, 0.82797464, -0.30620401, 0.16003707, -1.44411381, 0.87616892, -0.50446586, 0.23009474, 0.76201118]]
# yhat = model.predict(row)
# print('Prediction: %.3f' % yhat[0])

1
X Shape is  (295040, 1)
Y Shape is  (295040, 1)
X Shape is  (295040, 1)
Y Shape is  (295040, 1)


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
# omit_lowest = 20
# _selected = list(top_features[omit_lowest:].index)

# Building the Keras Model

In [36]:
df_baseline = df_Injury_End_fillna.copy()
sort_by = 'PLAYER_ID'

In [37]:
tf.keras.backend.set_floatx('float64')

# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)


x = df_baseline
X = X.values.reshape(-1, 1)

# x = tf.ones((3, 3))
Y = model(X)

print(Y)

tf.Tensor(
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [9.71412435e+01 2.03400406e+01 1.08151455e+02 1.48975821e+02]
 [6.83131107e-01 1.43038260e-01 7.60558756e-01 1.04764993e+00]
 ...
 [3.41565554e-01 7.15191301e-02 3.80279378e-01 5.23824967e-01]
 [3.38149898e+00 7.08039388e-01 3.76476584e+00 5.18586718e+00]
 [4.78191775e-01 1.00126782e-01 5.32391130e-01 7.33354954e-01]], shape=(295040, 4), dtype=float64)


Insert Layers

In [38]:
df_baseline = df_Injury_End_fillna.copy()
sort_by = 'PLAYER_ID'

In [39]:
# Create 3 layers
layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

# Call layers on a test input
X = df_baseline
X = X.values.reshape(-1, 1)

Y = layer3(layer2(layer1(X)))
print(Y)

tf.Tensor(
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 ...
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]], shape=(295040, 4), dtype=float64)


# Part 3 Conclusions

This is where the conclusions section will be typed